In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob, os
from sdt_dask.clients.azure.azure_client import AzureClient
from sdt_dask.dataplugs.pvdb_plug import PVDBPlug
from sdt_dask.dask_tool.runner import Runner

# Import user-defined dataplug

## Pull from redshift database

In [3]:
PVDBPlug?

Init signature: PVDBPlug(power_col='meas_val_f')
Docstring:      Dataplug class for retrieving data from the PVDB (Redshift) database.
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dataplugs/pvdb_plug.py
Type:           type
Subclasses:     

In [4]:
redshift_data_plug = PVDBPlug()

KEYS = [("TABJC1027159", 0), ("TAAI01129193", 0)]

redshift_data_plug.get_data(KEYS[0])

,ac_power_01
ts,
2016-03-28 12:40:00,1.1394
2016-03-28 12:45:00,1.4464
2016-03-28 12:50:00,1.1930
2016-03-28 12:55:00,2.1952
2016-03-28 13:00:00,1.4514
...,...
2019-06-20 15:30:00,0.4866
2019-06-20 15:35:00,0.4978
2019-06-20 15:40:00,0.4789


# Initialize the Client

In [7]:
# using the maximum memory limit for Azure VM
client_setup = AzureClient(
    workers=1,
    threads=2,
    memory=15.63,
    resource_group="pvinsight-dask",
    vnet="dask-vnet",
    security_group="pvinsight-dask",
    docker_image="nimishy/p_3.10.11_dask:latest",
    location="westus2",
    vm_size="Standard_D4s_v3",
    public_ingress=True,
    disk_size=100,
)
# Dask Local Client Initialization
client = client_setup.init_client()

Initializing Azure Cluster with 1 workers, 2 threads and 15.63MiB per worker...


FileNotFoundError: [Errno 2] No such file or directory: 'az'

# Use Runner to take in data plug and cluster then run the Dask tasks


In [ ]:
runner = Runner(data_plug=redshift_data_plug,
                client=client,
                output_path=f"../result/")

runner.set_up(KEYS, fix_shifts=True, verbose=True)
runner.visualize()

from IPython.display import display, Image
display(Image(filename='sdt_graph.png'))


# Get final result table

In [ ]:
runner.get_result()

# The result table and dask report has been saved in the folder you specified in the output_path parameter of the Runner object,
# you could check it out